In [2]:
import socket
import json

# Public Fulcrum Host (Mainnet)
FULCRUM_HOST = "bch.imaginary.cash"
FULCRUM_PORT = 50002 # This is usually the SSL port; use 50001 for TCP

def fulcrum_request(method, params):
    """Sends a JSON-RPC request to the Fulcrum server."""
    payload = json.dumps({"method": method, "params": params, "id": 1}) + "\n"
    with socket.create_connection((FULCRUM_HOST, 50001)) as sock:
        sock.sendall(payload.encode())
        response = sock.recv(65536) # Buffer for large NFT lists
        return json.loads(response.decode())

def get_token_details(category_id):
    print(f"--- Fetching Data for Category: {category_id[:10]}... ---")

    # 1. Get Genesis Transaction (for Supply & Issuer)
    gen_tx = fulcrum_request("blockchain.transaction.get", [category_id, True])
    
    # 2. Get Owners List
    owners = fulcrum_request("blockchain.token.get_owners", [category_id])
    
    # 3. Get NFT List
    nfts = fulcrum_request("blockchain.token.get_nft_list", [category_id])

    # --- Processing Data ---
    
    # Genesis Info
    issuer_addr = gen_tx['result']['vin'][0].get('address', 'Unknown')
    outputs = gen_tx['result']['vout']
    genesis_supply = sum(int(o['tokenData']['amount']) for o in outputs if 'tokenData' in o and 'amount' in o)
    
    # NFT Info
    nft_count = len(nfts.get('result', []))
    is_minting = any(n.get('capability') == 'minting' for n in nfts.get('result', []))

    # Output Report
    print(f"\n[ GENESIS INFO ]")
    print(f"Genesis TXID:    {category_id}")
    print(f"Owner (Issuer):  {issuer_addr}")
    print(f"Initial Supply:  {genesis_supply}")
    
    print(f"\n[ TOKEN STATUS ]")
    print(f"Total NFTs:      {nft_count}")
    print(f"Active Minting:  {'Yes' if is_minting else 'No'}")
    
    print(f"\n[ TOP HOLDERS ]")
    # Show first 5 owners as a sample
    for owner in owners.get('result', [])[:5]:
        print(f"Address: {owner['address']} | Amount: {owner['amount']}")

# --- EXECUTION ---
# Replace with the Category ID you want to check
target_category = "9c8362ec067e2d516064b6184b6ef0c9a6e5daa7dfb4693e9764de48460b3d9b"
get_token_details(target_category)

--- Fetching Data for Category: 9c8362ec06... ---


JSONDecodeError: Unterminated string starting at: line 1 column 2867 (char 2866)

In [1]:
import socket
import json

# Recommended Public Hosts
# bch.imaginary.cash | fulcrum.greyh.at | electroncash.de
HOST = "bch.imaginary.cash"
PORT = 50001 

def fulcrum_call(method, params):
    try:
        with socket.create_connection((HOST, PORT), timeout=10) as sock:
            payload = json.dumps({"method": method, "params": params, "id": 1}) + "\n"
            sock.sendall(payload.encode())
            
            # Larger buffer for tokens with many holders
            data = b""
            while True:
                chunk = sock.recv(4096)
                data += chunk
                if b'\n' in chunk: break
            return json.loads(data.decode())
    except Exception as e:
        return {"error": str(e)}

def get_token_report(cat_id):
    print(f"--- Analyzing Category: {cat_id[:10]}... ---")

    # 1. Get Genesis Transaction
    tx_data = fulcrum_call("blockchain.transaction.get", [cat_id, True])
    if "error" in tx_data:
        print("Error: Could not find transaction.")
        return

    # 2. Get Current Owners (Rich List)
    owners_data = fulcrum_call("blockchain.token.get_owners", [cat_id])
    
    # 3. Get All NFTs
    nft_data = fulcrum_call("blockchain.token.get_nft_list", [cat_id])

    # --- PROCESSING ---
    result = tx_data.get('result', {})
    
    # Identify Issuer (The address that funded the genesis)
    issuer = result.get('vin', [{}])[0].get('address', 'Unknown/Script')

    # Calculate Genesis Supply (Summing all token outputs in the first TX)
    gen_fungible_supply = 0
    for out in result.get('vout', []):
        t_data = out.get('tokenData')
        if t_data and 'amount' in t_data:
            gen_fungible_supply += int(t_data['amount'])

    # Rich List Calculation
    holders = owners_data.get('result', [])
    total_current_supply = sum(int(h['amount']) for h in holders)
    
    # NFT Calculation
    nfts = nft_data.get('result', [])
    minting_capable = any(n.get('capability') == 'minting' for n in nfts)

    # --- HUMAN READABLE OUTPUT ---
    print(f"\n[ IDENTITY ]")
    print(f"Category ID:   {cat_id}")
    print(f"Issuer Address: {issuer}")
    
    print(f"\n[ GENESIS STATS ]")
    print(f"Initial Fungible Supply: {gen_fungible_supply:,}")
    
    print(f"\n[ CURRENT STATE ]")
    print(f"Current Fungible Supply: {total_current_supply:,}")
    print(f"Total Unique NFT UTXOs:  {len(nfts)}")
    print(f"Active Minting NFT?      {'YES' if minting_capable else 'NO'}")
    
    print(f"\n[ TOP 5 HOLDERS ]")
    if not holders:
        print("No fungible holders found (this might be an NFT-only category).")
    else:
        for i, h in enumerate(holders[:5], 1):
            print(f"{i}. {h['address']} -> {int(h['amount']):,}")

# Test with your ID
category = "9c8362ec067e2d516064b6184b6ef0c9a6e5daa7dfb4693e9764de48460b3d9b"
get_token_report(category)

--- Analyzing Category: 9c8362ec06... ---

[ IDENTITY ]
Category ID:   9c8362ec067e2d516064b6184b6ef0c9a6e5daa7dfb4693e9764de48460b3d9b
Issuer Address: Unknown/Script

[ GENESIS STATS ]
Initial Fungible Supply: 0

[ CURRENT STATE ]
Current Fungible Supply: 0
Total Unique NFT UTXOs:  0
Active Minting NFT?      NO

[ TOP 5 HOLDERS ]
No fungible holders found (this might be an NFT-only category).


In [5]:
import socket
import json

def fulcrum_query(method, params):
    # Connecting to a reliable public Fulcrum server
    with socket.create_connection(("bch.imaginary.cash", 50001)) as sock:
        payload = json.dumps({"method": method, "params": params, "id": 1}) + "\n"
        sock.sendall(payload.encode())
        full_res = b""
        while True:
            chunk = sock.recv(1024*1024) # Large buffer for rich list
            full_res += chunk
            if b'\n' in chunk: break
        return json.loads(full_res.decode())

cat_id = "3f3a3250babfd578b7d4fc90defb11475c0c9dca9e1e9739af15321da89e43e7"

# 1. Get Current Holders & Sum Supply
holders_res = fulcrum_query("blockchain.token.get_owners", [cat_id])
all_holders = holders_res.get('result', [])
total_raw_supply = sum(int(h['amount']) for h in all_holders)

# 2. Get Genesis to find the Original Creator
gen_tx = fulcrum_query("blockchain.transaction.get", [cat_id, True])
creator_address = gen_tx['result']['vin'][0].get('address', 'Unknown')

print(f"--- FULCRUM REPORT ---")
print(f"Category: {cat_id}")
print(f"Original Creator: {creator_address}")
print(f"Raw Supply (Satoshis): {total_raw_supply:,}")
print(f"Human Supply (If 8 decimals): {total_raw_supply / 10**8:,.2f}")
print(f"Total Unique Holders: {len(all_holders)}")

--- FULCRUM REPORT ---
Category: 3f3a3250babfd578b7d4fc90defb11475c0c9dca9e1e9739af15321da89e43e7
Original Creator: Unknown
Raw Supply (Satoshis): 0
Human Supply (If 8 decimals): 0.00
Total Unique Holders: 0
